In [96]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse as dateparse

In [52]:
payload = dict(building_url="the-promenade-west-lofts-condos-for-sale-lease-downtown-losangeles")

In [122]:
r = requests.post("https://www.dlxco.com/property/getproperty/name/R-Listing/value/240/building/28", data=payload)

In [123]:
soup = BeautifulSoup(r.text, "html5lib")

In [124]:
def parse_li(li):
    parts = li.a.text.split()
    return dict(
        url=li.a['href'],
        unit=int(parts[1]),
        price=int(parts[3].replace("$", "").replace(",", "")),
        date=dateparse(parts[4]),
        bedrooms=int(parts[5].split("/")[0].replace("BR", "")),
        bathrooms=int(parts[5].split("/")[1].replace("BATHS", "").replace(",", "")),
        square_feet=int(parts[6].replace(",", ""))
    )

In [125]:
df = pd.DataFrame([parse_li(li) for li in soup.find_all("li", {"class": "leased-prop"})])

In [126]:
df['price_per_square_foot'] = df['price'] / df['square_feet']

In [127]:
df.sort_values("date", ascending=False).to_csv("./_data/sales.csv", index=False)